In [110]:
from py2neo import *
import json
IP="bolt://db:7687"
USER="neo4j"
PASSWORD="pass"
graph_db = Graph(IP, auth=(USER, PASSWORD))

## Objet Param

In [111]:
class Param:
    def __init__(self,entree="",sortie="",longueur_max=5,limit=10):
        self.entree = entree
        self.sortie = sortie
        self.longueur_max = longueur_max
        self.limit = limit
        
    def set_Entree(self,entree):
        self.entree = entree
        
    def set_Sortie(self,sortie):
        self.sortie = sortie
    
    def set_Longueur_max(self,longueur_max):
        self.longueur_max = longueur_max
    
    def set_Limit(self, limit):
        self.limit = limit
        


In [112]:
p1 = Param("Sequence set", "Alignment", 5,100)

print(p1.entree)
print(p1.sortie)
print(p1.longueur_max)
print(p1.limit)

Sequence set
Alignment
5
100


In [113]:
p2 = Param(0,"Alignment",0,100)

## Pour creer la requete

In [114]:
class Requete:
    def __init__(self,param):
        self.requete = ""
        self.param = param
        
    def creer_Requete(self):
        #entree
        self.requete = self.requete + f"""MATCH path=(i:IO {{term: '{self.param.entree}'}})"""
        #longueur_max
        self.requete = self.requete + f"""-[:inputOf|outputOf *1..{self.param.longueur_max}]->"""
        #sortie
        self.requete = self.requete + f"""(o:IO {{term: '{self.param.sortie}'}}) \n"""
        #du blabla pour pas que ca boucle
        self.requete = self.requete + """ WHERE NONE (n IN nodes(path) WHERE size([x IN nodes(path) WHERE n = x]) > 1 ) \n RETURN path AS p ORDER BY length(path) ASC\n"""
        #limit
        self.requete = self.requete + f"""LIMIT {self.param.limit}"""
    
        return self.param

In [115]:
r1 = Requete(p1)
r1.creer_Requete()

In [116]:
r = r1.requete

In [117]:
graph_db.run(r)

p
(_10411)-[:inputOf {}]->(_83988)-[:outputOf {}]->(_10790)
(_10411)-[:inputOf {}]->(_81550)-[:outputOf {}]->(_10335)-[:inputOf {}]->(_83649)-[:outputOf {}]->(_10790)
(_10411)-[:inputOf {}]->(_92092)-[:outputOf {}]->(_11051)-[:inputOf {}]->(_82165)-[:outputOf {}]->(_10790)


In [118]:
print(r1.requete)

MATCH path=(i:IO {term: 'Sequence set'})-[:inputOf|outputOf *1..5]->(o:IO {term: 'Alignment'}) 
 WHERE NONE (n IN nodes(path) WHERE size([x IN nodes(path) WHERE n = x]) > 1 ) 
 RETURN path AS p ORDER BY length(path) ASC
LIMIT 100
